## Visualizing mean wages weighted by the local cost of living

This weekend I participated in the hackathon [Hack For Sweden 2018](https://hackforsweden.se/). It is focused on using open data from the swedish government agencies to come up with ideas and applications for improving the environment, general well-being of the population, the job market, safety, etc.

My team developed a framework to identify the best areas in the country to settle if one wants to work remotely. One part of this concept required the relative quantification of wages in each County, meaning that a fair comparison of wages had to take into consideration the local cost of living. This way, the optimal place for working would be where wages were high and cost of living low. We called the wage normalized with the cost of living, *Smart Cash Index* (SCI).

This seemed like a good small project for dealing with messy structured data and learning a bit of geocoding and mapping quantities.

### Sources for the data

* Local cost of living: [Statistika Centralbyrån](http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__HE__HE0202/HE0202T06/table/tableViewLayout1/?rxid=ddccda22-c8fc-461a-9d50-c27bf861c804) (SCB)
* Mean wages for each County: [Lönestatistik.se](http://www.lonestatistik.se/loner.asp/yrke/Programmerare-1001) (in this analysis, we used the wages for programming jobs)
* Population of each municipality: [Wikipedia](https://en.wikipedia.org/wiki/List_of_municipalities_of_Sweden) (needed for applying the cost of living information)

### The code

In [1]:
import os
import pandas as pd

folder = 'smart_cash_data'

# Loading local cost of living data
living_cost = pd.read_csv(os.path.join(folder, 'cost_of_living.csv'))
living_cost['2015'] = (living_cost['2015']*1000/12).astype(int)
living_cost.columns = ['Location', 'Cost of living (2015)']
living_cost

,Location,Cost of living (2015)
0,00 Riket,20783
1,0010 Stor-Stockholm,23791
2,0020 Stor-Göteborg,20650
3,0042 Kommuner med > 75000 inv (exkl Stor-Stock...,19258
4,0043 Kommuner med < 75000 inv (exkl Stor-Stock...,20516


In [2]:
# Loading mean wages for each County
wages = pd.read_csv(os.path.join(folder, 'it_wages.csv'))
wages['Medellön'] = wages['Medellön'].str.replace(' kr', '').apply(pd.to_numeric)
wages = wages[['Län', 'Medellön']]
wages.columns = ['County', 'Mean wage']
wages

,County,Mean wage
0,Blekinge,29.140
1,Dalarna,29.000
2,Gotland,25.825
3,Gävleborg,25.529
4,Halland,27.418
5,Jämtland,32.913
6,Jönköping,32.982
7,Kalmar,31.088
8,Kronoberg,29.843
9,Norrbotten,28.560


I used [wikitable2csv](http://wikitable2csv.ggor.de/) for converting the population info from a wikipedia table to a .csv file.

In [3]:
# Loading and preprocessing local population information. 
muni = pd.read_csv(os.path.join(folder, 'municipalities.csv'))
muni['County'] = muni['County'].str.replace(' County', '')
muni['Municipality'] = muni['Municipality'].str.replace(' Municipality', '')
muni = muni[['Municipality', 'County', 'Population']]
muni.sample(10)

,Municipality,County,Population
231,Trollhättan,Västra Götaland,56393
120,Lidingö,Stockholm,45034
138,Mark,Västra Götaland,33689
198,Stockholm,Stockholm,894165
267,Åmål,Västra Götaland,12203
55,Gällivare,Norrbotten,18380
257,Vårgårda,Västra Götaland,11066
155,Norrköping,Östergötland,133444
50,Gnosjö,Jönköping,9390
159,Nykvarn,Stockholm,9502


In [4]:
# Applying local cost of living information for each municipality
muni.loc[:, 'Cost of living'] = 20516
muni.loc[muni['Population'] > 75000, 'Cost of living'] = 19258
muni.loc[muni['County'] == 'Stockholm', 'Cost of living'] = 23791
muni.loc[muni['Municipality'] == 'Göteborg', 'Cost of living'] = 20650

# Filling wages values for each County
muni['Wages'] = 0
for wage in wages['County']:
    muni.loc[muni['County'] == wage, 'Wages'] = wages.loc[wages['County'] == wage, 'Mean wage'].values

# Smart cash is the wage normalized by the consumption index or cost of living
muni['smart_cash'] = muni['Wages']/muni['Cost of living']

# Mean by County and rescaling the smart_cash
smart_cash = muni.groupby('County')['smart_cash'].mean().reset_index()
spread = (smart_cash['smart_cash'].max() - smart_cash['smart_cash'].min())
smart_cash['smart_cash_idx'] = (smart_cash['smart_cash'] - smart_cash['smart_cash'].min()) / spread
smart_cash.drop('smart_cash', axis=1, inplace=True)

In [5]:
smart_cash

,County,smart_cash_idx
0,Blekinge,0.390359
1,Dalarna,0.374491
2,Gotland,0.014647
3,Gävleborg,0.000000
4,Halland,0.262856
5,Jämtland,0.817978
6,Jönköping,0.844582
7,Kalmar,0.611139
8,Kronoberg,0.497652
9,Norrbotten,0.339726


### Visualization

A choroplath map is what we need here. A [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) is a thematic map in which areas (in this case, Counties) are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map. Here, the variable is of course the SCI.

Besides the data, we need the coordinates of each County, in the [geojson](http://geojson.org/) format. The choropleth is built with the [folium package](http://python-visualization.github.io/folium/docs-v0.5.0/), that gives interactive plots and it's extremely easy to use.

In [6]:
import folium

geo = os.path.join(folder, 'sweden-counties.geojson')

m = folium.Map(location=[62, 18], width='60%', zoom_start=5, detect_retina=True, tiles='Mapbox Bright')

m.choropleth(
    geo_data=geo,
    name='Smart Cash',
    data=smart_cash,
    columns=['County', 'smart_cash_idx'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    highlight=True,
    legend_name=''
)


folium.LayerControl().add_to(m)

m

Here, `0` indicates the County where the mean wage is lowest with respect to the local cost of living. `1` is the oposite, where the wage represents the maximum purchasing power.

This is the end of this post! Let me know what you think in the comments.

In [7]:
# import os
# import subprocess
# outdir = "./" # this directory has to exist..
# m.save("tmp.html")
# url = "file://{}/tmp.html".format(os.getcwd())
# outfn = os.path.join(outdir,"outfig.png")
# subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])

0